In [1]:
import os
import pandas as pd

In [2]:
temprature_measurments = "temprature measurments.xlsx"
mean_hourly_tempratures = "meanhour.xlsx"

In [3]:
def parsfile(filepath):
    f = open(filepath)
    line = []
    for i,l in enumerate(f):
        if i == 0 :
            continue
        if i in (1,2):
            res =[a.strip() for a in l.split(":")]
            line.append(res[1])
        if i in (3,4):
            res = [a.strip().replace(":","") for a in l.split("  ") if a.strip() != ""]
            line.append(res[1])
            line.append(res[3])
    f.close()
    return line
        

In [4]:
def parse_folder(dir, time_offset_minute=0):
    files = os.listdir(dir)
    dic = {}
    for p in files:
        if p.endswith(".txt"):
            dic[p] = parsfile(os.path.join(dir, p))

    columns=["2.5um", "10um", "AT", "RH", "DP", "WB"]
    df = pd.DataFrame.from_dict(dic, orient="index", columns=columns).reset_index()
    for col in columns:
        df[col] = df[col].astype(float)
    df.rename(columns={"index":"timestamp"}, inplace=True)
    df["timestamp"] = df["timestamp"].str.replace(".txt", "")
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%m-%d-%Y %H-%M-%S")
    df["timestamp"] = df["timestamp"] + pd.Timedelta(minutes=time_offset_minute)
    df["hour"] = df["timestamp"].dt.time
    df["timestamp_hour"] = df["timestamp"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
    
    return df

In [5]:
df1 = parse_folder("./2022/19072022", time_offset_minute=0)

In [6]:
df2 = parse_folder("./2022/19072022+1", time_offset_minute=1)

In [7]:
df3 = parse_folder("./2022/19072022+5", time_offset_minute=5)

In [8]:
df4 = parse_folder("./2022/19072022+6", time_offset_minute=6)

In [9]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [10]:
df

,timestamp,2.5um,10um,AT,RH,DP,WB,hour,timestamp_hour
0,2022-07-19 10:17:45,5.0,27.0,26.3,39.7,12.9,18.6,10:17:45,2022-07-19 10:00:00
1,2022-07-19 10:18:47,9.0,33.0,27.3,37.4,13.0,19.1,10:18:47,2022-07-19 10:00:00
2,2022-07-19 10:19:49,5.0,18.0,27.7,36.9,13.2,19.3,10:19:49,2022-07-19 10:00:00
3,2022-07-19 10:20:51,4.0,21.0,27.9,40.5,14.5,20.0,10:20:51,2022-07-19 10:00:00
4,2022-07-19 10:21:54,5.0,30.0,28.0,35.8,13.1,19.4,10:21:54,2022-07-19 10:00:00
...,...,...,...,...,...,...,...,...,...
498,2022-07-19 16:00:40,4.0,13.0,37.0,16.1,11.5,22.7,16:00:40,2022-07-19 16:00:00
499,2022-07-19 16:01:43,4.0,11.0,37.0,15.6,11.2,22.6,16:01:43,2022-07-19 16:00:00
500,2022-07-19 16:02:45,3.0,6.0,37.0,15.8,11.4,22.7,16:02:45,2022-07-19 16:00:00
501,2022-07-19 16:03:48,3.0,10.0,37.2,15.9,11.6,22.8,16:03:48,2022-07-19 16:00:00


In [11]:
mean_temperatures = df.groupby("timestamp_hour", as_index=False).agg("mean")

In [12]:
df.to_excel(temprature_measurments, index=False)

In [13]:
mean_temperatures["hour"] = mean_temperatures["timestamp_hour"].dt.time

In [14]:
mean_temperatures

,timestamp_hour,2.5um,10um,AT,RH,DP,WB,hour
0,2022-07-19 10:00:00,4.828571,22.571429,29.365714,34.017143,13.588571,20.151429,10:00:00
1,2022-07-19 11:00:00,5.622642,23.396226,31.715094,28.630189,13.537736,21.067925,11:00:00
2,2022-07-19 12:00:00,4.243243,17.027027,32.678378,23.413514,11.970270,20.975676,12:00:00
3,2022-07-19 13:00:00,3.960000,12.580000,35.570000,18.940000,12.064000,22.202000,13:00:00
4,2022-07-19 14:00:00,4.078947,12.289474,36.410526,17.310526,11.794737,22.494737,14:00:00
5,2022-07-19 15:00:00,3.588235,9.970588,36.855882,15.391176,10.964706,22.514706,15:00:00
6,2022-07-19 16:00:00,3.838710,10.677419,37.725806,14.564516,11.109677,22.929032,16:00:00
7,2022-07-19 17:00:00,3.500000,9.115385,38.176923,13.803846,10.953846,23.092308,17:00:00
8,2022-07-19 18:00:00,3.820513,9.487179,37.797436,14.307692,11.000000,22.930769,18:00:00
9,2022-07-19 19:00:00,3.794872,14.923077,36.228205,19.107692,12.707692,22.630769,19:00:00


In [15]:
mean_temperatures.to_excel(mean_hourly_tempratures, index=False)